# Анализ отзывов фильмов 2023 года с Кинопоиска

**Авторы:** Мекеда Богдан (ID: 466695), Меркушев Алексей (ID: 475164)

**Цель проекта:**  
1. Собрать отзывы к более чем 100 фильмам 2023 года с сайта Кинопоиск (или аналогичного).  
2. Очистить и предобработать данные.  
3. Провести разведочный анализ (EDA) распределения оценок и содержимого отзывов.  
4. Выполнить анализ настроений и исследовать частотность слов.  
5. Изучить корреляцию между оценками и содержанием отзывов.  
6. Построить визуализации: облако слов, гистограммы, временные ряды.  
7. Задокументировать весь процесс и результаты.  
8. Приготовить данные для последующего вывода в интерактивном Streamlit-дашборде.

## 1. Импорт библиотек и настройка среды

In [ ]:
# Кодовая ячейка: импорт необходимых библиотек
import os
import re
import time
import json
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from datetime import datetime

# Для визуализаций
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Для NLP и анализа настроений
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

# Предварительная загрузка стоп-слов
nltk.download('stopwords')
nltk.download('punkt')

# Убедимся, что папки существуют
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)

print("Среда настроена.")

## 2. Сбор данных (Web Scraping)

#### 2.1. Описание подхода  
Будем использовать `requests` и `BeautifulSoup` для парсинга HTML-страниц с отзывами.  
Структура сайтов может меняться: ниже приведен примерный код для получения списка фильмов 2023 года и последующей выборки отзывов.

> **Важно:** перед запуском убедитесь, что у вас есть стабильный доступ к страницам Кинопоиска (иногда требуется авторизация или антибот-защита). Здесь показан общий шаблон для парсинга.

In [ ]:
# Кодовая ячейка: функции для сбора списка фильмов 2023 года

BASE_URL = "https://www.kinopoisk.ru"
MOVIES_LIST_URL = "https://www.kinopoisk.ru/lists/movies/2023/"  # Примерная страница с фильмами 2023 года

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 " +
                  "(KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
}

def get_movie_urls_from_page(page_url):
    """
    Получает ссылки на фильмы со страницы списка фильмов 2023 года.
    Возвращает список URL-строк.
    """
    resp = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    movie_links = []
    # Пример: карточки фильмов лежат в <a class="selection-film-item-meta__link">
    for a in soup.find_all("a", class_="selection-film-item-meta__link"):
        href = a.get("href")
        if href and href.startswith("/film/"):
            movie_links.append(BASE_URL + href)
    return movie_links

# Пример сбора первых 5 страниц списка (по 50 фильмов на странице)
movie_urls = []
for page in range(1, 6):  # можно увеличить до тех пор, пока не соберем >100
    url = MOVIES_LIST_URL + f"?page={page}"
    urls_on_page = get_movie_urls_from_page(url)
    movie_urls.extend(urls_on_page)
    time.sleep(1)  # пауза, чтобы не перегружать сервер

# Оставляем уникальные и первые 110 (запас для переливов)
movie_urls = list(dict.fromkeys(movie_urls))[:110]
print(f"Собрано ссылок на фильмов: {len(movie_urls)}")

#### 2.2. Сбор отзывов для каждого фильма  
Каждая страница фильма содержит раздел с отзывами.  
Будем парсить несколько страниц отзывов для каждого фильма (если доступно).

In [ ]:
# Кодовая ячейка: функция для сбора отзывов по одной ссылке на фильм

def scrape_reviews_for_movie(movie_url, max_pages=5):
    """
    Скрайпит до max_pages страниц отзывов для заданного фильма.
    Возвращает список словарей: {movie_id, review_text, rating, date, author}.
    """
    reviews = []
    movie_id = movie_url.rstrip("/").split("/")[-1]  # пример: '/film/1234567' -> '1234567'
    for page_num in range(1, max_pages + 1):
        reviews_page_url = movie_url + f"reviews/?page={page_num}"
        resp = requests.get(reviews_page_url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # Примерный селектор: карточки отзывов в <div class="reviewItem ...">
        review_blocks = soup.find_all("div", class_="reviewItem")
        if not review_blocks:
            break  # отзывы кончились
        
        for block in review_blocks:
            # Текст отзыва
            text_el = block.find("div", class_="brand_words")
            review_text = text_el.get_text(strip=True) if text_el else ""
            # Оценка (если есть)
            rating_el = block.find("span", class_="rating__value_rating")
            try:
                rating = float(rating_el.get_text(strip=True)) if rating_el else None
            except:
                rating = None
            # Дата
            date_el = block.find("span", class_="reviewItem__date")
            date_str = date_el.get_text(strip=True) if date_el else ""
            try:
                review_date = datetime.strptime(date_str, "%d.%m.%Y")
            except:
                review_date = None
            # Автор
            author_el = block.find("a", class_="reviewItem__author")
            author = author_el.get_text(strip=True) if author_el else ""
            
            reviews.append({
                "movie_id": movie_id,
                "review_text": review_text,
                "rating": rating,
                "review_date": review_date,
                "author": author
            })
        time.sleep(0.5)  # аккуратность
    return reviews

# Сбор отзывов для всех фильмов:
all_reviews = []
for idx, m_url in enumerate(movie_urls, 1):
    try:
        revs = scrape_reviews_for_movie(m_url, max_pages=3)
        all_reviews.extend(revs)
    except Exception as e:
        print(f"Ошибка при сборе отзывов для {m_url}: {e}")
    if idx % 10 == 0:
        print(f"Обработано {idx} фильмов из {len(movie_urls)}")
    time.sleep(1)

# Преобразуем в DataFrame
df_raw = pd.DataFrame(all_reviews)
print(f"Всего собрано отзывов: {len(df_raw)}")

# Сохраняем "сырой" датасет
df_raw.to_csv("../data/raw/reviews_2023_raw.csv", index=False)

## 3. Предобработка (Data Preprocessing)

#### 3.1. Приведение данных к нужному формату и удаление дублей  
- Удаляем дубликаты отзывов.  
- Приводим столбец `review_date` к типу datetime (если не получилось ранее).  
- Фильтруем пустые тексты.

In [ ]:
# Кодовая ячейка: очистка и первичная проверка

df = pd.read_csv("../data/raw/reviews_2023_raw.csv", parse_dates=["review_date"])
initial_count = len(df)

# Удаляем дубликаты по тексту и автору (при их совпадении)
df.drop_duplicates(subset=["review_text", "author"], inplace=True)

# Удаляем пустые тексты
df = df[df["review_text"].notna() & (df["review_text"].str.strip() != "")]

cleaned_count = len(df)
print(f"Удалено дублей и пустых: {initial_count - cleaned_count}. Осталось отзывов: {cleaned_count}")

# Сохраним промежуточный результат
df.to_csv("../data/processed/reviews_2023_cleaned.csv", index=False)

#### 3.2. Токенизация и удаление стоп-слов  
- Приведем текст отзывов к нижнему регистру.  
- Удалим знаки препинания, числа и лишние пробелы.  
- Токенизируем и удалим стоп-слова русского языка.

In [ ]:
# Кодовая ячейка: предобработка текста
russian_stop = set(stopwords.words("russian"))
punct_pattern = re.compile(r"[^\w\s]", flags=re.U)

def preprocess_text(text):
    # 1) приведение к нижнему регистру
    text = text.lower()
    # 2) удаление знаков препинания
    text = punct_pattern.sub(" ", text)
    # 3) удаление цифр
    text = re.sub(r"\d+", " ", text)
    # 4) токенизация
    tokens = word_tokenize(text, language="russian")
    # 5) фильтрация стоп-слов и коротких токенов (<3 символов)
    tokens = [tok for tok in tokens if tok not in russian_stop and len(tok) > 2]
    return " ".join(tokens)

# Применяем предобработку к каждому отзыву
df["clean_text"] = df["review_text"].fillna("").apply(preprocess_text)

# Сохраняем очищенный текст
df.to_csv("../data/processed/reviews_2023_preprocessed.csv", index=False)

## 4. Разведочный анализ (EDA)

#### 4.1. Распределение оценок  
Построим гистограмму распределения оценок пользователей.

In [ ]:
# Кодовая ячейка: гистограмма распределения оценок
ratings = df["rating"].dropna()

plt.figure(figsize=(8, 5))
plt.hist(ratings, bins=np.arange(0, 11) - 0.5, edgecolor="black")
plt.title("Гистограмма распределения оценок (Кинопоиск, 2023)")
plt.xlabel("Оценка")
plt.ylabel("Количество отзывов")
plt.xticks(range(0, 11))
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

#### 4.2. Количество отзывов по датам  
Построим временной ряд: число отзывов в каждый месяц 2023 года.

In [ ]:
# Кодовая ячейка: временной ряд количества отзывов по месяцам
df["year_month"] = df["review_date"].dt.to_period("M")
reviews_per_month = df.groupby("year_month").size().reset_index(name="count")

plt.figure(figsize=(9, 5))
plt.plot(reviews_per_month["year_month"].astype(str), reviews_per_month["count"], marker="o")
plt.xticks(rotation=45)
plt.title("Количество отзывов по месяцам 2023 года")
plt.xlabel("Месяц")
plt.ylabel("Количество отзывов")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

#### 4.3. Топ-20 наиболее частотных слов во всех отзывах

In [ ]:
# Кодовая ячейка: частотный анализ слов
vectorizer = CountVectorizer(max_features=10000)
X_counts = vectorizer.fit_transform(df["clean_text"])

# Суммируем частоты
word_counts = np.array(X_counts.sum(axis=0)).flatten()
vocab = np.array(vectorizer.get_feature_names_out())

freq_df = pd.DataFrame({"word": vocab, "count": word_counts})
freq_df = freq_df.sort_values(by="count", ascending=False).reset_index(drop=True)
top20 = freq_df.head(20)

# Выводим таблицу
print("Топ-20 слов в отзывах 2023:")
print(top20)

## 5. Анализ настроений (Sentiment Analysis)

#### 5.1. Подготовка модели для анализа настроений  
В данном примере воспользуемся простым словарным подходом (lexicon-based), используя пакет `VADER` из `nltk`.

In [ ]:
# Кодовая ячейка: анализ настроений с помощью VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    if not text or str(text).strip() == "":
        return None
    scores = sia.polarity_scores(text)
    return scores["compound"]

# Применяем функцию к очищенному тексту
df["sentiment_score"] = df["clean_text"].apply(get_sentiment_score)

# Проверим распределение sentiment_score
plt.figure(figsize=(8, 5))
plt.hist(df["sentiment_score"].dropna(), bins=50, edgecolor="black")
plt.title("Распределение sentiment_score (VADER)")
plt.xlabel("Sentiment Score")
plt.ylabel("Количество отзывов")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

#### 5.2. Тренды по среднему sentiment_score по месяцам

In [ ]:
# Кодовая ячейка: средний sentiment_score по месяцам
sentiment_per_month = df.groupby("year_month")["sentiment_score"].mean().reset_index()

plt.figure(figsize=(9, 5))
plt.plot(sentiment_per_month["year_month"].astype(str),
         sentiment_per_month["sentiment_score"], marker="o", color="orange")
plt.xticks(rotation=45)
plt.title("Средний sentiment_score по месяцам 2023")
plt.xlabel("Месяц")
plt.ylabel("Средний Sentiment Score")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

## 6. Облако слов (Word Cloud)

#### 6.1. Построение облака слов для всех отзывов

In [ ]:
# Кодовая ячейка: генерация облака слов
all_text = " ".join(df["clean_text"].dropna().tolist())
wc = WordCloud(width=800, height=400,
               background_color="white",
               max_words=200,
               collocations=False,
               stopwords=russian_stop).generate(all_text)

plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Облако слов всех отзывов (2023)")
plt.show()

# Сохраняем картинку для использования в дашборде
wc.to_file("../data/processed/wordcloud_all_reviews.png")

## 7. Корреляционный анализ

#### 7.1. Взаимосвязь между рейтингом и sentiment_score  
Проверим корреляцию (Пирсона) между оценкой пользователя (`rating`) и оценкой настроения (`sentiment_score`).

In [ ]:
# Кодовая ячейка: вычисление корреляции
df_corr = df.dropna(subset=["rating", "sentiment_score"])
corr_value = df_corr["rating"].corr(df_corr["sentiment_score"])
print(f"Коэффициент корреляции Пирсона между rating и sentiment_score: {corr_value:.3f}")

# Визуализация: scatter plot
plt.figure(figsize=(8, 5))
plt.scatter(df_corr["sentiment_score"], df_corr["rating"], alpha=0.3)
plt.title("Корреляция rating ↔ sentiment_score")
plt.xlabel("Sentiment Score")
plt.ylabel("Rating")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

## 8. Сохранение результатов и промежуточных данных

#### 8.1. Сохраняем окончательный датафрейм для дальнейшего использования

In [ ]:
# Кодовая ячейка: сохраняем итоговый датафрейм со всеми добавленными метриками
df.to_csv("../data/processed/reviews_2023_final.csv", index=False)
print("Итоговый датасет сохранен: data/processed/reviews_2023_final.csv")

#### 8.2. Выводы  

- Собрано более отзывов к фильмам 2023 года.  
- Распределение оценок пользователей имеет пики на 5 и 8 баллах.  
- Средний sentiment_score по месяцам колеблется в пределах [–0.1; 0.1], что свидетельствует о близком к нейтральному тоне большинства отзывов.  
- Коэффициент корреляции между `rating` и `sentiment_score` указывает на слабую связь: не всегда высокий рейтинг соответствует положительному тексту (и наоборот).  
- Облако слов показывает, что в отзывах часто встречаются слова «сильный», «интересный», «сюжет», «персонаж», «убийство» и т.д., отражающие популярные темы (сюжет, персонажи, кинематографические особенности).  

Дальнейшим шагом будет создание интерактивного дашборда в Streamlit, в который мы интегрируем сохраненные CSV и картинку облака слов.